This demo provides a simple demonstration of how to load a Compute Graph (e.g., .pkl saved by AutoGO, or .pb file) and instantiate it as TensorFlow or PyTorch architecture.
We will rely on the CGs in architectures/samples for this demo

In [ ]:
# First load dependencies
import torch as t
import tensorflow as tf
import pickle
from model_src.comp_graph.tf_comp_graph import OP2I, ComputeGraph
from model_src.comp_graph.tf_comp_graph_utils import compute_cg_flops
from model_src.comp_graph.tf_comp_graph_output import CompGraphOutputNet as TFOutputNet
from model_src.comp_graph.tf_comp_graph_output_torch import CompGraphOutputNet as TorchOutputNet

op2i = OP2I().build_from_file()

"""
Define some helper functions for loading and inference
"""
def load_cg_pb(filename, cg_name="MyCG", res=[32, 32, 3]):
    assert filename.endswith(".pb")
    cg = ComputeGraph(name=cg_name, H=res[0], W=res[1], C_in=res[2])
    cg.build_from_pb(filename, op2i, oov_threshold=0.)
    return cg

def load_cg_pkl(filename):
    with open(filename, "rb") as f:
        cg = pickle.load(f)
    return cg 

def gen_as_tf_inference(cg, op2i, res=[32, 32, 3]):
    tf_net = TFOutputNet(op2i, cg)
    test_tensor = tf.random.uniform(shape=[1, res[0], res[1], res[2]])
    print(f"Test TensorFlow model with input: {test_tensor.shape}")
    print(f"Output shape: {tf_net(test_tensor).shape}")
    return tf_net

def gen_as_torch_inference(cg, op2i, res=[32, 32, 3]):
    torch_net = TorchOutputNet(op2i, cg)
    test_tensor = t.rand(1, res[2], res[0], res[1])
    print(f"Test PyTorch model with input: {test_tensor.shape}")
    print(f"Output shape: {torch_net(test_tensor).shape}")
    return torch_net


Let's now play around with the CIFAR-10 families.
Each family has a folder in `architectures/samples`. This folder contains:
- `input_arch.png` -> This is the original architecture we optimized with AutoGO. Its CG pkl file is in `architectures`, and this picture shows how it is segmented by our Byte-Pair Encoding DB.
- `best_autogo.pkl` -> CG file for the best architecture found by AutoGO (bold in Table 2)
- `best_autogo.png` -> Illustration of the best architecture segmented by our BPE Encoding DB.

In [ ]:
print("Testing CIFAR-10 CGs")
families = ['nb101', 'nb201', 'hiaml', 'inception', 'two_path']
for family in families:
    print(f"Test {family}")
    cg = load_cg_pkl(f"architectures/samples/{family}/best_autogo.pkl")
    cg.name = f"{family}_best"
    print(f"Architecture FLOPs: {compute_cg_flops(cg, op2i, use_fast_counter=True, div=1e6)}")
    print("Inference tests on TensorFlow and PyTorch")
    gen_as_tf_inference(cg, op2i)
    gen_as_torch_inference(cg, op2i)

Let's now play around with our VGG architecture:

In [ ]:
print("Test on VGG")
base_vgg = load_cg_pkl("architectures/samples/vgg/vgg16_bn.pkl")
print(f"Original Architecture FLOPs: {compute_cg_flops(base_vgg, op2i, use_fast_counter=True, div=1e9)}")
print(f"Original Architecture Nodes/Edges: {len(base_vgg.nodes)}/{len(base_vgg.edge_pairs)}")
autogo_vgg = load_cg_pkl("architectures/samples/vgg/vgg_autogo.pkl")
print(f"AutoGO Architecture FLOPs: {compute_cg_flops(autogo_vgg, op2i, use_fast_counter=True, div=1e9)}")
print(f"AutoGO Architecture Nodes/Edges: {len(autogo_vgg.nodes)}/{len(autogo_vgg.edge_pairs)}")
print("Inference tests on TensorFlow and PyTorch")
gen_as_tf_inference(autogo_vgg, op2i, res=[224, 224, 3])
gen_as_torch_inference(autogo_vgg, op2i, res=[224, 224, 3])

And finally, EDSR.
For this, we have 4 networks:
- The original EDSR encoder
- EDSR AutoGO Arch 1/2/3

For the mutated architecture, `_changes.png` is an image which highlights the subgraph segments present in the mutated model but are **not** in the original model.

In [ ]:
print("Test on EDSR")
# Base EDSR architecture is in .pb format, so we load differently
base_edsr = load_cg_pb("architectures/samples/edsr/edsr_encoder.pb", cg_name="EDSR_Encoder", res=[320, 180, 64]) # Also uses a different size input, because it is the Encoder in the middle of a larger network.
print(f"Original Architecture FLOPs: {compute_cg_flops(base_edsr, op2i, use_fast_counter=True, div=1e9)}")
print(f"Original Architecture Nodes/Edges: {len(base_edsr.nodes)}/{len(base_edsr.edge_pairs)}")

edsr_arch1 = load_cg_pkl("architectures/samples/edsr/autogo_arch1.pkl")
print(f"Arch1 Architecture FLOPs: {compute_cg_flops(edsr_arch1, op2i, use_fast_counter=True, div=1e9)}")
print(f"Arch1 Architecture Nodes/Edges: {len(edsr_arch1.nodes)}/{len(edsr_arch1.edge_pairs)}")
print("Inference tests on TensorFlow and PyTorch")
gen_as_tf_inference(edsr_arch1, op2i, res=[320, 180, 64])
gen_as_torch_inference(edsr_arch1, op2i, res=[320, 180, 64])

edsr_arch2 = load_cg_pkl("architectures/samples/edsr/autogo_arch2.pkl")
edsr_arch3 = load_cg_pkl("architectures/samples/edsr/autogo_arch3.pkl")
print(f"Arch2 Architecture FLOPs: {compute_cg_flops(edsr_arch2, op2i, use_fast_counter=True, div=1e9)}")
print(f"Arch3 Architecture FLOPs: {compute_cg_flops(edsr_arch3, op2i, use_fast_counter=True, div=1e9)}")